In [2]:
import os, sys
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
sys.path.append('./s2spy')
# from s2spy import RGDR, time
import func_SPI, utils, h5py
path_obs_data = '/data/volume_2/observational/'
filename = 'chrips_precip_1981-2021.nc'
path_tp = os.path.join(path_obs_data, 'raw', filename)

In [2]:
dfs = pd.read_hdf('/data/volume_2/subseasonal/ecmwf/aggregated/aggregation_timestamps.h5').sort_index()
df = dfs[['aggregation_start_inclusive', 'aggregation_end_inclusive']].copy()
sel_months = [10, 11, 12]
mon_start_mask = [True if m in sel_months else False for m in df.set_index('aggregation_start_inclusive').index.month]
mon_end_mask = [True if m in sel_months else False for m in df.set_index('aggregation_end_inclusive').index.month]
mon_mask = np.logical_and(mon_start_mask, mon_end_mask)

In [3]:
df_sub = dfs[mon_mask][dfs[mon_mask].index.year==dfs[mon_mask].index.year[0]]
anchor_dates = [(d.month, d.day) for d in pd.to_datetime(df_sub['aggregation_end_inclusive'].values)]
# for anchor in anchor_dates:
anchor = anchor_dates[0]
calendar = time.AdventCalendar(anchor=anchor, freq=f'{int(7*4)}d', max_lag=1)
calendar.map_years(2000, 2000) ; calendar.get_intervals()

i_interval,0,1
anchor_year,,
2000,"(2000-10-01, 2000-10-29]","(2000-09-03, 2000-10-01]"


In [8]:
tp = xr.open_dataarray(path_tp)
tp = utils.formatting_ds(tp, format_lon='west_east')
tp = utils.get_selbox(tp, selbox=(33.12, 51.12, -4.625, 14.88))
append_intervals = []
for i, anchor in enumerate(anchor_dates):
    calendar = time.AdventCalendar(anchor=anchor, freq=f'{int(7*4)}d', max_lag=1)
    calendar = calendar.map_to_data(tp)
    tp_r = time.resample(calendar, tp).isel(i_interval=0)
    tp_r = tp_r.assign_coords({'i_interval':[i], # i_interval should be counter within target period
                               'index':np.arange(0,tp_r.anchor_year.size), # index should be a unique index for each interval
                              'interval':tp_r.interval.values.astype(str)}) # pd.IntervalDtype cannot be store as netcdf, converting to str dtype
    tp_r = tp_r.reset_index(('index', 'interval'))
    tp_r = tp_r.drop(('index', 'interval'))
    append_intervals.append(tp_r)

tp_aggr = xr.concat([_.rename({'i_interval':'target_period'}) for _ in append_intervals], dim='target_period')
tp_aggr = tp_aggr.transpose('anchor_year', 'target_period', 'latitude', 'longitude')
tp_aggr.to_netcdf('/data/volume_2/observational/preprocessed/chrips_precip_1981-2021_africa_aggregated_intervals.nc')

PermissionError: [Errno 13] Permission denied: b'/data/volume_2/observational/preprocessed/chrips_precip_1981-2021_africa_aggregated_intervals.nc'

In [9]:
tp_aggr.to_netcdf('/data/volume_2/observational/preprocessed/chrips_precip_1981-2021_africa_aggregated_intervals.nc')

In [ ]:
aggr = 1
path_spi = filename.split('_')[0] + f'_SPI_{aggr}_' + '_'.join(filename.split('_')[2:])
output = func_SPI.calc_SPI_from_daily(tp.to_array(), aggr)

In [ ]:
output.to_netcdf(os.path.join(path_obs_data, 'preprocessed', path_spi))

In [ ]:
latitude = 0 ; longitude=30
ts_raw = tp.to_array().sel(latitude=latitude, longitude=longitude, method='nearest')
ts_stn = output.sel(latitude=latitude, longitude=longitude, method='nearest')
# ts_pack = SMI_package.sel(latitude=40, longitude=240)

fig = plt.figure(figsize=(20,10) )

# plot observed versus corresponding Gamma probability
ax1 = plt.subplot(1, 2, 1)
vals = ax1.plot(ts_raw, '.k');
# ax1.set_ylabel('Cumulative probability (from Gamma distribution)')
# ax1.set_xlabel('Aggregated Precipitation [mm] - Original values')
# plot transformed standard normal from Gamma probability
ax1 = plt.subplot(1, 2, 2)
vals = ax1.plot(ts_stn, '.k');
# ax1.plot(ts_pack, '.r')
# ax1.set_ylabel('Cumulative probability (from Gamma distribution)')
ax1.set_xlabel('SPI - Gamma prob. transformed to standard normal ')